In [ ]:
import os
from dotenv import load_dotenv
from dune_client.client import DuneClient
import pandas as pd

load_dotenv()
dune = DuneClient(os.getenv('DUNE_API_KEY'))

def unpack(query_result):
    if query_result and query_result.result and query_result.result.rows:
        df = pd.DataFrame(query_result.result.rows)
    else:
        print("No data available in the query result.")
        df = pd.DataFrame()  # Create an empty DataFrame if no data is available
    return df

corn_df = unpack(dune.get_latest_result(4014290))
corn_df.to_csv('Corn Trades.csv')
wallet_analysis = corn_df['wallet_address'].value_counts().reset_index()
wallet_analysis.columns = ['wallet_address', 'transaction_count']


In [ ]:
poster_df = unpack(dune.get_latest_result(4013378))

poster_df

In [ ]:
import os
import json
import asyncio
from ring_doorbell import Ring, Auth
from oauthlib.oauth2 import MissingTokenError

cache_file = "token.cache"

def token_updated(token):
    with open(cache_file, "w") as f:
        f.write(json.dumps(token))

def otp_callback():
    auth_code = input("2FA code: ")
    return auth_code

async def main():
    if os.path.isfile(cache_file):
        with open(cache_file) as f:
            token = json.load(f)
    else:
        token = None

    auth = Auth("MyProject/1.0", token, token_updated)

    try:
        await asyncio.to_thread(auth.fetch_token, os.getenv('RING_USERNAME'), os.getenv('RING_PASSWORD'))
    except MissingTokenError:
        await asyncio.to_thread(auth.fetch_token, os.getenv('RING_USERNAME'), os.getenv('RING_PASSWORD'), otp_callback())

    ring = Ring(auth)
    await ring.update_data()

    devices = ring.devices()
    for device in devices['doorbots']:
        print('Doorbot name: {}'.format(device.name))
        print('Battery: {}'.format(device.battery_life))
        print('Wifi: {}'.format(device.wifi_signal_strength))

def run_main():
    asyncio.run(main())

if __name__ == "__main__":
    try:
        run_main()
    except RuntimeError as e:
        if str(e) == "asyncio.run() cannot be called from a running event loop":
            loop = asyncio.get_running_loop()
            loop.create_task(main())
        else:
            raise